# Otimização de Rede Neural com Gradiente Descendente ($b_3$)

utilizaremos o **Gradiente Descendente** para refinar o ajuste final de uma rede neural. Com o objetivo de encontrar o valor ideal para o viés da camada de saída (**$b_3$**), mantendo os outros pesos fixos.

A estrutura matemática da rede (Forward Pass) é definida por:

$$
h_1 = \ln(1 + e^{(x \cdot w_1 + b_1)})
$$

$$
h_2 = \ln(1 + e^{(x \cdot w_2 + b_2)})
$$

$$
\hat{y} = (h_1 \cdot w_3 + h_2 \cdot w_4) + \mathbf{b_3}
$$

**Onde:**
* **$x$**: Dado de entrada.
* **$h_1, h_2$**: Neurônios da camada oculta (ativados pela função **Softplus**).
* **$w_1...w_4$**: Pesos da rede (que manteremos congelados/fixos).
* **$b_3$**: **Viés Final** (o parâmetro que vamos otimizar dinamicamente).
* **$\hat{y}$**: A previsão final da rede.

O algoritmo calculará o gradiente (a direção do erro) considerando **todo o conjunto de dados** a cada iteração, ajustando $b_3$ para minimizar a diferença entre a curva prevista ($\hat{y}$) e os dados reais.

##Bibliotecas que usaremos




In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

## Definição das variaveis com os dados que usaremos:

In [2]:
w1, w2, w3, w4 = 3.34, -3.53, -1.22, -2.3
b1, b2 = -1.43, 0.57
b3 = 0
values_of_b3 = []
list_current_gradient = []
list_sum_of_squared_residuals = []

x_data = np.array([0, 0.5, 1])
y_data = np.array([0, 1, 0])
max_iterations = 1000
precision = 0.0001
learning_rate = 0.01

# Definição das funções
Para facilitar a leitura, vamos associar as variáveis do código aos símbolos matemáticos:

* **$x$**: `input` (Dados de entrada)
* **$w_1, w_2$**: Pesos da primeira camada
* **$b_1, b_2$**: Vieses da primeira camada
* **$h_1, h_2$**: `y_curve1`, `y_curve2` (Saída dos neurônios da camada oculta, após ativação)
* **$w_3, w_4$**: Pesos da camada de saída
* **$b_3$**: Viés final
* **$\hat{y}$**: `y_curve` / `predictions` (Previsão final)



## 1. Equação da Rede Neural (Forward Pass)
Correspondente à função `neural_network`.

* Cálcula a ativação dos neurônios ocultos usando a função **Softplus** ($\ln(1+e^z)$)
* Combina os resultados linearmente.

$$
h_1 = \ln(1 + e^{(x \cdot w_1 + b_1)})
$$

$$
h_2 = \ln(1 + e^{(x \cdot w_2 + b_2)})
$$

**A saída final é:**

$$
\hat{y} = (h_1 \cdot w_3 + h_2 \cdot w_4) + b_3
$$


In [3]:
def neural_network(input, b3):
  y_curve1 = np.log(1 + np.exp(input*w1+b1))
  y_curve2 = np.log(1 + np.exp(input*w2+b2))
  y_curve = (y_curve1*w3+y_curve2*w4) +b3
  return y_curve


## 2. Derivada da Função de Custo
Correspondente à função `calculate_derivative`.

Derivada da Soma dos Quadrados dos Resíduos ($SSR$) em relação ao parâmetro $b_3$.

$$
\frac{\partial SSR}{\partial b_3} = \sum_{i=1}^{n} -2 (y_i - \hat{y}_i)
$$

In [4]:
def calculate_derivative(b3):
  predictions = neural_network(x_data, b3)
  residuals = y_data - predictions
  return np.sum(-2 * residuals)

## Loop de Treinamento: Otimizando $b_3$

A rede neural aprende o melhor valor para o viés final ($b_3$) repetindo o seguinte ciclo:

1.  **Forward Pass:** A rede faz uma previsão com o valor atual de $b_3$.
2.  **Cálculo do Erro:** Medimos a distância entre a previsão e o valor real (SSR).
3.  **Backward Pass:** Calculamos o gradiente (`calculate_derivative`) para descobrir a direção do ajuste.
4.  **Atualização:** Modificamos $b_3$ usando a taxa de aprendizado.
5. **Verifica a Precisão e se os passos de atualização para o $b_3$forem pequenos demais**

O processo se repete até que o gradiente seja próximo de zero (convergência) ou o maximo de iterações for atingido.

Também armazenamos o histórico dos valores de $b_3$ para a visualização da convergência do algoritmo em gráficos posteriores.

In [5]:
for i in range(max_iterations):
    values_of_b3.append(b3)

    predictions = neural_network(x_data, b3)
    sum_of_squared_residuals = np.sum((y_data - predictions)**2)
    list_sum_of_squared_residuals.append(sum_of_squared_residuals)

    current_gradient = calculate_derivative(b3)
    list_current_gradient.append(current_gradient)

    print(f"""Step Size: {(learning_rate * current_gradient):.4f}
Old b3: {b3:.4f}
New b3: {(b3 - learning_rate * current_gradient):.4f}\n""",
              flush=True)

    b3 = b3 - learning_rate * current_gradient
    if np.abs(current_gradient*learning_rate) < precision:
        break

print(f"b3 Final: {b3:.4f}")
print(f"gradiente Final: {current_gradient:.4f}")
print(f"Total de Iterações: {len(values_of_b3)}")

Step Size: -0.1566
Old b3: 0.0000
New b3: 0.1566

Step Size: -0.1472
Old b3: 0.1566
New b3: 0.3037

Step Size: -0.1383
Old b3: 0.3037
New b3: 0.4420

Step Size: -0.1300
Old b3: 0.4420
New b3: 0.5721

Step Size: -0.1222
Old b3: 0.5721
New b3: 0.6943

Step Size: -0.1149
Old b3: 0.6943
New b3: 0.8092

Step Size: -0.1080
Old b3: 0.8092
New b3: 0.9172

Step Size: -0.1015
Old b3: 0.9172
New b3: 1.0187

Step Size: -0.0954
Old b3: 1.0187
New b3: 1.1141

Step Size: -0.0897
Old b3: 1.1141
New b3: 1.2038

Step Size: -0.0843
Old b3: 1.2038
New b3: 1.2882

Step Size: -0.0793
Old b3: 1.2882
New b3: 1.3674

Step Size: -0.0745
Old b3: 1.3674
New b3: 1.4419

Step Size: -0.0700
Old b3: 1.4419
New b3: 1.5120

Step Size: -0.0658
Old b3: 1.5120
New b3: 1.5778

Step Size: -0.0619
Old b3: 1.5778
New b3: 1.6397

Step Size: -0.0582
Old b3: 1.6397
New b3: 1.6978

Step Size: -0.0547
Old b3: 1.6978
New b3: 1.7525

Step Size: -0.0514
Old b3: 1.7525
New b3: 1.8039

Step Size: -0.0483
Old b3: 1.8039
New b3: 1.8522



### Animação da Convergência (Ajuste de $b_3$)

O código a seguir gera a visualização do treinamento da **Rede Neural** otimizando apenas o viés b3.

A função de atualização (`update`) reconstrói a curva de previsão somando o viés atualizado ao resultado fixo das camadas anteriores:

$$
\hat{y} = \underbrace{(h_1 \cdot w_3 + h_2 \cdot w_4)}_{\text{Camadas Ocultas}} + b_{3(\text{atual})}
$$

In [6]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(x_data, y_data, color='blue', zorder=5, label='Pontos Originais')
ax.set_title('Gradiente Descendente: Convergência do Parâmetro b3')
ax.set_xlabel('Eixo X')
ax.set_ylabel('Eixo Y')
ax.legend()
ax.grid(True)

x_curve_anim = np.linspace(min(x_data) - 0.2, max(x_data) + 0.2, 100)

curve, = ax.plot(x_curve_anim, neural_network(x_curve_anim, values_of_b3[0]), color='red', label='Curva da Rede Neural')
iteration_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, color='green')

def update(frame):
    current_b3 = values_of_b3[frame]
    y_curve_anim = neural_network(x_curve_anim, current_b3)
    curve.set_ydata(y_curve_anim)
    iteration_text.set_text(f'Iteração: {frame+1}\nb3: {current_b3:.4f}')
    return curve, iteration_text,

ani = FuncAnimation(fig, update, frames=len(values_of_b3), interval=80, blit=True)

plt.close(fig)
HTML(ani.to_jshtml())

Output hidden; open in https://colab.research.google.com to view.

### Visualizando a Função de Custo e a Tangente

* **Curva Roxa:** Representa o erro total (SSR) para diferentes valores de $b_3$. Nosso objetivo é chegar ao fundo desse vale.
* **Linha Laranja (Tangente):** Visualiza a derivada no ponto atual.

Equação da reta tangente:

$$
y = \text{gradiente} \cdot (x - b_3) + \text{SSR}
$$

**Onde:**
* **$y$** : O valor da linha tangente que será desenhado
* **$b_3$**: A posição atual no eixo X.
* **SSR**: A altura atual no eixo Y (o erro).
* **Gradiente**: A inclinação da reta ($m$).


In [7]:
intercept_values_plot = np.linspace(0, 5, 200)

sum_of_squared_values_plot = [np.sum((y_data - neural_network(x_data, b3_val))**2) for b3_val in intercept_values_plot]

fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(intercept_values_plot, sum_of_squared_values_plot, color='purple', linestyle='-', label='Função de Perda (SSR)')
ax.set_title('Convergência na Função de Perda com Tangente')
ax.set_xlabel('Intercept (b3)')
ax.set_ylabel('Soma dos Quadrados dos Resíduos (SSR)')
ax.legend()
ax.grid(True)

point, = ax.plot([], [], 'o', color='red', markersize=8, label='Intercepto Atual')
tangent_line, = ax.plot([], [], '--', color='orange', label='Linha Tangente')
iteration_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, color='green', fontsize=12)

def update(frame):
    if frame >= len(values_of_b3) or frame >= len(list_sum_of_squared_residuals) or frame >= len(list_current_gradient):
        return point, tangent_line, iteration_text

    current_intercept = values_of_b3[frame]
    current_ssr = list_sum_of_squared_residuals[frame]
    current_gradient = list_current_gradient[frame]

    point.set_data([current_intercept], [current_ssr])

    tangent_x = np.linspace(current_intercept - 0.2, current_intercept + 0.2, 2)
    tangent_y = current_gradient * (tangent_x - current_intercept) + current_ssr

    tangent_line.set_data(tangent_x, tangent_y)

    iteration_text.set_text(f'Iteração: {frame+1}\nIntercepto: {current_intercept:.4f}\nSSR: {current_ssr:.4f}')

    return point, tangent_line, iteration_text

ani = FuncAnimation(fig, update, frames=len(values_of_b3), interval=80, blit=True)

plt.close(fig)
HTML(ani.to_jshtml())


Output hidden; open in https://colab.research.google.com to view.